**IMPORT LIBRARIES**

In [ ]:
import time
import google.generativeai as genai
from google.colab import userdata

**Google API Key Configuration and Model Initialization**

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

**Context Window and API Retry Configuration**

In [ ]:
CONTEXT_WINDOW_MAX = 1000000  # Maximum context capacity in tokens
TOKEN_SAFETY_MARGIN = CONTEXT_WINDOW_MAX * 0.8  # Warning threshold at 80% capacity
RETRY_ATTEMP_MAX = 3  # Maximum retries for API calls
BASE_RETRY_DELAY = 2  # Initial delay between retries in seconds

**Conversation History and Token Tracking**

In [ ]:
conversation_log = []  # Stores conversation history
cumulative_tokens = 0  # Tracks total tokens consumed

def tokenCount(input_text):
    """Calculates token count for input text using model's tokenizer."""
    try:
        return model.count_tokens(input_text).total_tokens
    except Exception as error:
        print(f"Token counting error: {error}")
        return 0

**Text Generation for Retry Mechanism and Token Tracking**

In [ ]:
def generate_text(dialog_history):
    """Generates text response using API with retry mechanism and token tracking."""
    global cumulative_tokens
    attempt_count = 0
    retry_wait = BASE_RETRY_DELAY

    while attempt_count < RETRY_ATTEMP_MAX:
        try:
            response = model.generate_content(dialog_history)

            if response and hasattr(response, "usage_metadata"):
                # Extract token usage data from response metadata
                usage_data = response.usage_metadata
                cumulative_tokens = usage_data.total_token_count
                prompt_usage = usage_data.prompt_token_count

                print(f"Token Usage - Total: {cumulative_tokens}, Prompt: {prompt_usage}")
            else:
                print("API response missing usage metadata")
                cumulative_tokens = 0

            return response

        except Exception as api_error:
            # Handle rate limiting with exponential backoff
            if "rate limit" in str(api_error).lower() or "429" in str(api_error):
                print(f"Rate limited. Retrying in {retry_wait}s (Attempt {attempt_count+1}/{RETRY_ATTEMP_MAX})")
                time.sleep(retry_wait)
                attempt_count += 1
                retry_wait *= 1.5  # Increase wait time by 50% each retry
            else:
                print(f"API communication failed: {api_error}")
                return None

    print("Maximum retry attempts exhausted. Service unavailable.")
    return None

**Conversation Processing with Context Management and Token Constraints**

In [ ]:
def process_message(user_input):
    """Manages conversation flow and context window constraints."""
    global conversation_log
    conversation_log.append(f"User: {user_input}")

    # Context window management loop
    while True:
        current_dialog = "\n".join(conversation_log)
        token_usage = tokenCount(current_dialog)

        # Remove older messages when exceeding context limits
        if token_usage > CONTEXT_WINDOW_MAX:
            # Maintain conversation pairs by removing oldest interactions
            removable_items = 2 if len(conversation_log) >= 2 else 1
            conversation_log = conversation_log[removable_items:]
        else:
            # Display warning when approaching context limit
            if token_usage >= TOKEN_SAFETY_MARGIN:
                print(f"⚠️ Context window at {token_usage}/{CONTEXT_WINDOW_MAX} tokens")
            break

    bot_response = generate_text(current_dialog)

    if bot_response:
        conversation_log.append(f"Assistant: {bot_response.text}")
        return bot_response.text
    return None

**Context Window Enforcement Testing**

In [ ]:
def test_context_window():
    """Validates context window limit enforcement mechanism."""
    global conversation_log
    conversation_log = []
    sample_message = ("Repeating context test pattern " * 30000).strip()

    print("\n=== Initiating Context Window Test ===")

    try:
        threshold_80 = int(CONTEXT_WINDOW_MAX * 0.8)  # 80% threshold
        warned_80 = False  # Track if 80% warning was issued

        while tokenCount("\n".join(conversation_log)) < CONTEXT_WINDOW_MAX:
            # Simulate conversation history growth
            conversation_log.extend([
                f"User: {sample_message}",
                "Assistant: Test response"
            ])
            current_tokens = tokenCount("\n".join(conversation_log))
            print(f"Current token count: {current_tokens}")

            # Warn when crossing 80% threshold (once)
            if not warned_80 and current_tokens >= threshold_80:
                print("-- Warning: Approached 80% of context window")
                warned_80 = True

            # Existing safety margin check
            if current_tokens >= TOKEN_SAFETY_MARGIN:
                print("-- Safety threshold notification verified")

        print("✔️ Context limit enforcement operational")

    except Exception as test_error:
        print(f"Context test failure: {test_error}")

    print("=== Context Test Complete ===\n")

**API Rate Test**

In [ ]:
def test_rate_limit():
    """Evaluates API rate limit handling capabilities."""
    global conversation_log, cumulative_tokens
    test_payload = "Stress test message " * 100  # High token content

    print("Initializing rate limit evaluation...")
    print(f"Baseline token count: {cumulative_tokens}")

    while cumulative_tokens < CONTEXT_WINDOW_MAX:
        api_result = process_message(test_payload)
        if not api_result:
            print("Terminating test due to API failure")
            break

        print("API request successful, continuing stress test...")
        time.sleep(0.15)  # Brief pause between requests

    print("Rate limit evaluation cycle completed")

**Interactive Chat Method**

In [ ]:
def chat():
    """Main interactive chat interface."""
    while True:
        user_message = input("You: ")

        if user_message.lower() in ['exit', 'quit']:
            print("Closing chat session...")
            break

        response = process_message(user_message)

        if response:
            print(f"Assistant: {response}")
        else:
            print("Response generation failed. Please retry.")

**Input Text Processing with Token Limit Enforcement**

In [ ]:
def text(input_text):
    """Processes input text after multiplying its length by 20,000, with token limit enforcement."""
    expanded_text = input_text * 400000  # Multiply input text length
    token_usage = tokenCount(expanded_text)

    if token_usage >= CONTEXT_WINDOW_MAX:
        return "❌ Input exceeds maximum token limit. Please shorten your text."

    if token_usage >= TOKEN_SAFETY_MARGIN:
        return f"⚠️ Warning: Your input is {token_usage}/{CONTEXT_WINDOW_MAX} tokens. Consider shortening it."

    return process_message(expanded_text)

**Test Chat Mode**

In [ ]:
chat()

You: hello
Token Usage - Total: 14, Prompt: 3
Assistant: Hello there! How can I help you today?

You: how are you
Token Usage - Total: 42, Prompt: 22
Assistant: Assistant: I'm doing well, thank you for asking!  How are you today?



KeyboardInterrupt: Interrupted by user

**Testing Rate Limit**

In [ ]:
test_rate_limit()

Initializing rate limit evaluation...
Baseline token count: 42
Token Usage - Total: 424, Prompt: 348
API request successful, continuing stress test...
Token Usage - Total: 794, Prompt: 730
API request successful, continuing stress test...
Token Usage - Total: 1105, Prompt: 1100
API request successful, continuing stress test...
Token Usage - Total: 1418, Prompt: 1411
API request successful, continuing stress test...
Token Usage - Total: 1731, Prompt: 1724
API request successful, continuing stress test...
Token Usage - Total: 2044, Prompt: 2037
API request successful, continuing stress test...
Token Usage - Total: 2357, Prompt: 2350
API request successful, continuing stress test...
Token Usage - Total: 2670, Prompt: 2663
API request successful, continuing stress test...
Token Usage - Total: 2983, Prompt: 2976
API request successful, continuing stress test...
Token Usage - Total: 3296, Prompt: 3289
API request successful, continuing stress test...
Token Usage - Total: 3609, Prompt: 3602
A

Rate limited. Retrying in 2s (Attempt 1/3)


Rate limited. Retrying in 3.0s (Attempt 2/3)


Rate limited. Retrying in 4.5s (Attempt 3/3)
Maximum retry attempts exhausted. Service unavailable.
Terminating test due to API failure
Rate limit evaluation cycle completed


**Testing Context Window**

In [ ]:
test_context_window()


=== Initiating Context Window Test ===
Token counting error: Invalid input: 'content' argument must not be empty. Please provide a non-empty value.
Current token count: 150007
Current token count: 300015
Current token count: 450023
Current token count: 600031
Current token count: 750039
Current token count: 900047
-- Warning: Approached 80% of context window
-- Safety threshold notification verified
Current token count: 1050055
-- Safety threshold notification verified
✔️ Context limit enforcement operational
=== Context Test Complete ===



**Testing One Input for Token Limit Enforcement**

In [ ]:
text("naber")

'⚠️ Warning: Your input is 800000/1000000 tokens. Consider shortening it.'